# File to run MILC on COBRE for classification using pre-trained model

Import required lib and functions

In [1]:
import time
from collections import deque
from itertools import chain

import numpy as np
import torch
import sys
import os

from src.utils import get_argparser
from src.encoders_ICA import NatureCNN, NatureOneCNN
from src.slstm_attn_catalyst import LSTMTrainer
from src.lstm_attn import subjLSTM
from src.All_Architecture import combinedModel
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import h5py

/Users/umahmood1/opt/anaconda3/envs/MILC/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/Users/umahmood1/opt/anaconda3/envs/MILC/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/Users/umahmood1/opt/anaconda3/envs/MILC/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/Users/umahmood1/opt/anaconda3/envs/MILC/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



Parse the required arguments, add default values if not given

In [2]:
parser = get_argparser()
args = parser.parse_args()
tags = ['pretraining-only']
config = {}
config.update(vars(args))

/Users/umahmood1/opt/anaconda3/envs/MILC/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



Function to get indices for both classes in labels.

In [3]:
def find_indices_of_each_class(all_labels):
    HC_index = (all_labels == 0).nonzero()
    SZ_index = (all_labels == 1).nonzero()
    return HC_index, SZ_index

Intialize variables and values

In [4]:
start_time = time.time()
# do stuff

# ID = args.script_ID + 3
ID = args.script_ID - 1
JobID = args.job_ID
# ID = 2
print('ID = ' + str(ID))
print('exp = ' + args.exp)
print('pretraining = ' + args.pre_training)
sID = str(ID)
currentDT = datetime.datetime.now()
d1 = currentDT.strftime("%Y-%m-%d%H:%M:%S")
d2 = str(JobID) + '_' + str(ID)

Name = args.exp + '_FBIRN_' + args.pre_training
dir = 'run-' + d1 + d2 + Name
dir = dir + '-' + str(ID)
wdb = 'wandb_new'
output_path = "Output"
opath = os.path.join(os.getcwd(), output_path)
args.path = opath

wdb1 = 'wandb_new'
wpath1 = os.path.join(os.getcwd(), wdb1)

tfilename = str(JobID) + 'outputFILE' + Name + str(ID)

output_path = os.path.join(os.getcwd(), 'Output')
output_path = os.path.join(output_path, tfilename)

ntrials = 1
ngtrials = 10
tr_sub = [15, 25, 40]
sub_per_class = tr_sub[ID]
sample_x = 100
sample_y = 20
subjects = 157
tc = 140
# samples_per_subject = int(tc / sample_y)
samples_per_subject = 7
# samples_per_subject = int((tc - sample_y)+1)
# ntest_samples = 32
window_shift = 20
n_val_HC = 14
n_val_SZ = 16
n_test_HC = 14
n_test_SZ = 16
if torch.cuda.is_available():
    cudaID = str(torch.cuda.current_device())
    device = torch.device("cuda:" + cudaID)
    #device = torch.device("cuda:" + str(1))
else:
    device = torch.device("cpu")
print('device = ', device)
# Milc default
if args.exp == 'FPT':
    gain = [0.1, 0.05, 0.05]  # FPT
elif args.exp == 'UFPT':
    gain = [0.05, 0.45, 0.65]  # UFPT
else:
    gain = [0.25, 0.35, 0.65]  # NPT

current_gain = gain[ID]
args.gain = current_gain

ID = 0
exp = UFPT
pretraining = milc
device =  cpu


Import data, labels, pre calculated training indices to get same training samples everytime, file used to permute indices.

In [5]:
hf = h5py.File('../Data/COBRE_AllData.h5', 'r')
data = hf.get('COBRE_dataset')
data = np.array(data)
data = data.reshape(subjects, sample_x, tc)

# Get Training indices for cobre and convert them to tensor. this is to have same training samples everytime.
hf_hc = h5py.File('../IndicesAndLabels/COBRE_HC_TrainingIndex.h5', 'r')
HC_TrainingIndex = hf_hc.get('HC_TrainingIndex')
HC_TrainingIndex = np.array(HC_TrainingIndex)
HC_TrainingIndex = torch.from_numpy(HC_TrainingIndex)

hf_sz = h5py.File('../IndicesAndLabels/COBRE_SZ_TrainingIndex.h5', 'r')
SZ_TrainingIndex = hf_sz.get('SZ_TrainingIndex')
SZ_TrainingIndex = np.array(SZ_TrainingIndex)
SZ_TrainingIndex = torch.from_numpy(SZ_TrainingIndex)

if args.fMRI_twoD:
    finalData = data
    finalData = torch.from_numpy(finalData).float()
    finalData = finalData.permute(0, 2, 1)
    finalData = finalData.reshape(finalData.shape[0], finalData.shape[1], finalData.shape[2], 1)
else:
    finalData = np.zeros((subjects, samples_per_subject, sample_x, sample_y))
    for i in range(subjects):
        for j in range(samples_per_subject):
            finalData[i, j, :, :] = data[i, :, (j * window_shift):(j * window_shift) + sample_y]
    finalData = torch.from_numpy(finalData).float()

print(finalData.shape)
filename = '../IndicesAndLabels/correct_indices_GSP.csv'
# print(filename)
df = pd.read_csv(filename, header=None)
c_indices = df.values
c_indices = torch.from_numpy(c_indices).int()
c_indices = c_indices.view(53)
c_indices = c_indices - 1
finalData2 = finalData[:, :, c_indices.long(), :]

filename = '../IndicesAndLabels/index_array_labelled_COBRE.csv'
df = pd.read_csv(filename, header=None)
index_array = df.values
index_array = torch.from_numpy(index_array).long()
index_array = index_array.view(subjects)

filename = '../IndicesAndLabels/labels_COBRE.csv'
df = pd.read_csv(filename, header=None)
df = pd.read_csv(filename, header=None)
all_labels = df.values
all_labels = torch.from_numpy(all_labels).int()
all_labels = all_labels.view(subjects)
all_labels = all_labels - 1
finalData2 = finalData2[index_array, :, :, :]
all_labels = all_labels[index_array]

torch.Size([157, 7, 100, 20])


Call the indices function to get indices for each class


In [6]:
HC_index, SZ_index = find_indices_of_each_class(all_labels)

Break data into training, val and test datasets.

In [7]:
total_HC_index_tr = HC_index[:len(HC_index) - (n_val_HC + n_test_HC)]
total_SZ_index_tr = SZ_index[:len(SZ_index) - (n_val_SZ + n_test_SZ)]


HC_index_val = HC_index[len(HC_index) - (n_val_HC + n_test_HC):len(HC_index) - n_test_HC]
SZ_index_val = SZ_index[len(HC_index) - (n_val_SZ + n_test_SZ):len(HC_index) - n_test_SZ]

HC_index_test = HC_index[len(HC_index) - (n_test_HC):]
SZ_index_test = SZ_index[len(SZ_index) - (n_test_SZ):]

Define arrays to store output results

In [8]:
results = torch.zeros(ntrials, 4)
auc = torch.zeros(ntrials, 1)

Function to create outputfile and store progress

In [9]:
def output_file():
    # Output file to store the progress
    print('trial = ', trial)
    output_text_file = open(output_path, "a+")
    output_text_file.write("Trial = %d gTrial = %d\r\n" % (trial, 1))
    output_text_file.close()    
    

Get samples_per_subject for training and required labels

In [10]:
def samples_and_labels():
    # Get subject_per_class number of training samples


    HC_random = HC_TrainingIndex[trial]
    SZ_random = SZ_TrainingIndex[trial]
    HC_random = HC_random[:sub_per_class]
    SZ_random = SZ_random[:sub_per_class]
    # Choose the subject_per_class indices from HC_index_val and SZ_index_val using random numbers

    HC_index_tr = total_HC_index_tr[HC_random]
    SZ_index_tr = total_SZ_index_tr[SZ_random]


    # Join dataset of both classes
    tr_index = torch.cat((HC_index_tr, SZ_index_tr))
    val_index = torch.cat((HC_index_val, SZ_index_val))
    test_index = torch.cat((HC_index_test, SZ_index_test))

    tr_index = tr_index.view(tr_index.size(0))
    val_index = val_index.view(val_index.size(0))
    test_index = test_index.view(test_index.size(0))

    # Get the dataset and labels using indices created
    tr_eps = finalData2[tr_index, :, :, :]
    val_eps = finalData2[val_index, :, :, :]
    test_eps = finalData2[test_index, :, :, :]

    tr_labels = all_labels[tr_index.long()]
    val_labels = all_labels[val_index.long()]
    test_labels = all_labels[test_index.long()]
    return tr_eps, val_eps, test_eps, tr_labels, val_labels, test_labels

Define the model using encoder type (2DCNN or 1DCNN). We use 1DCNN for MILC. encoder and lstm is defined first and then passed to combinedModel which then adds attention and classifier on top. args.complete_arc = true means that path used for importing representation has a file best_full consisting of complete model (encoder + lstm + attention) (based on catalyst) where as false means path has seperate files for encoder, lstm and attention (Based on old code which didn't use catalyst). 

In [11]:
def define_model():
    
    
    if args.encoder_type == "Nature":
        encoder = NatureCNN(observation_shape[2], args)

    elif args.encoder_type == "NatureOne":
        dir = ""
        if args.pre_training == "basic":
            dir = 'PreTrainedEncoders/Basic_Encoder'
        elif args.pre_training == "milc":
            args.oldpath = wpath1 + '/PreTrainedEncoders/Milc'

        encoder = NatureOneCNN(observation_shape[2], args)
        lstm_model = subjLSTM(device, args.feature_size, args.lstm_size, num_layers=args.lstm_layers,
                              freeze_embeddings=True, gain=current_gain)

    complete_model = combinedModel(encoder, lstm_model, gain=current_gain, PT=args.pre_training,
                                   exp=args.exp, device=device, oldpath=args.oldpath,
                                   complete_arc=args.complete_arc)
    return complete_model

Define trainer based on the training required. sub-lstm is for downstream task and sub-enc-lstm is for pre-training.

In [12]:
def define_trainer():
    if args.method == "sub-lstm":
        trainer = LSTMTrainer(complete_model, config, device=device, tr_labels=tr_labels,
                              val_labels=val_labels, test_labels=test_labels, wandb="wandb", trial=str(trial))
        return trainer
    elif args.method == "sub-enc-lstm":
        print("Change method to sub-lstm")
        assert False
    else:
        assert False, "method {} has no trainer".format(args.method)
    

Create samples, models and trainers ntrials time and call the function pre-train while saving the results

In [17]:
    

    

    

    sss=0

    for trial in range(ntrials):
        observation_shape = finalData2.shape
        output_file()    
        tr_eps, val_eps, test_eps, tr_labels, val_labels, test_labels = samples_and_labels()
        complete_model = define_model()
        trainer = define_trainer()
        config = {}
        config.update(vars(args))
        config['obs_space'] = observation_shape
        results[trial][0], results[trial][1], results[trial][2], results[trial][3] = trainer.pre_train(tr_eps,
                                                                                                               val_eps,
                                                                                                               test_eps)

    

trial =  0
Complete Arch Loaded

1/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s]

/Users/umahmood1/opt/anaconda3/envs/MILC/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.




1/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.467, loss=4.811]
1/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.48it/s, accuracy01=0.467, loss=4.811]
1/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.39it/s, accuracy01=0.467, loss=4.811]

1/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s]
1/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.600, loss=4.696]
1/3000 * Epoch (valid): 100% 1/1 [00:00<00:00, 12.32it/s, accuracy01=0.600, loss=4.696]
[2020-10-02 17:27:11,503] 
1/3000 * Epoch 1 (train): accuracy01=0.4667 | auc/_mean=0.3844 | auc/class_0=0.3911 | auc/class_1=0.3778 | loss=4.8115
1/3000 * Epoch 1 (valid): accuracy01=0.6000 | auc/_mean=0.5647 | auc/class_0=0.7098 | auc/class_1=0.4196 | loss=4.6962
[2020-10-02 17:27:11,503] 
1/3000 * Epoch 1 (train): accuracy01=0.4667 | auc/_mean=0.3844 | auc/class_0=0.3911 | auc/class_1=0.3778 | loss=4.8115
1/3000 * Epoch 1 (valid): accuracy01=0.6000 | auc/_mean=0.5647 | auc/class_0=0.7098 | auc/class_1=0.4196 | loss=4

[2020-10-02 17:27:14,704] 
8/3000 * Epoch 8 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=3.8655
8/3000 * Epoch 8 (valid): accuracy01=0.5667 | auc/_mean=0.7701 | auc/class_0=0.7723 | auc/class_1=0.7679 | loss=4.0268

9/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s]
9/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s, accuracy01=1.000, loss=3.726]
9/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.62it/s, accuracy01=1.000, loss=3.726]
9/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.49it/s, accuracy01=1.000, loss=3.726]

9/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s]
9/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.567, loss=3.940]
9/3000 * Epoch (valid): 100% 1/1 [00:00<00:00, 11.35it/s, accuracy01=0.567, loss=3.940]
[2020-10-02 17:27:15,158] 
9/3000 * Epoch 9 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=3.7259
9/3000 * Epoch 9 (valid): accuracy01=0.5667 | auc/_mean=0.7790 

16/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s]
16/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.633, loss=3.392]
16/3000 * Epoch (valid): 100% 1/1 [00:00<00:00, 11.54it/s, accuracy01=0.633, loss=3.392]
[2020-10-02 17:27:18,257] 
16/3000 * Epoch 16 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=2.9701
16/3000 * Epoch 16 (valid): accuracy01=0.6333 | auc/_mean=0.7924 | auc/class_0=0.7857 | auc/class_1=0.7991 | loss=3.3917
[2020-10-02 17:27:18,257] 
16/3000 * Epoch 16 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=2.9701
16/3000 * Epoch 16 (valid): accuracy01=0.6333 | auc/_mean=0.7924 | auc/class_0=0.7857 | auc/class_1=0.7991 | loss=3.3917

17/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s]
17/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s, accuracy01=1.000, loss=2.890]
17/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.68it/s, accuracy01=1.000, loss=2.890]
17/3000 * Epoch (tr

...

123/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.54it/s, accuracy01=1.000, loss=0.120]
123/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.43it/s, accuracy01=1.000, loss=0.120]

123/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s]
123/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.633, loss=0.822]
123/3000 * Epoch (valid): 100% 1/1 [00:00<00:00, 11.05it/s, accuracy01=0.633, loss=0.822]
[2020-10-02 17:28:08,657] 
123/3000 * Epoch 123 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=0.1205
123/3000 * Epoch 123 (valid): accuracy01=0.6333 | auc/_mean=0.7991 | auc/class_0=0.7991 | auc/class_1=0.7991 | loss=0.8217
[2020-10-02 17:28:08,657] 
123/3000 * Epoch 123 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=0.1205
123/3000 * Epoch 123 (valid): accuracy01=0.6333 | auc/_mean=0.7991 | auc/class_0=0.7991 | auc/class_1=0.7991 | loss=0.8217

124/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it

[2020-10-02 17:28:11,832] 
130/3000 * Epoch 130 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=0.1000
130/3000 * Epoch 130 (valid): accuracy01=0.5667 | auc/_mean=0.8013 | auc/class_0=0.8036 | auc/class_1=0.7991 | loss=0.9944

131/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s]
131/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s, accuracy01=1.000, loss=0.098]
131/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.47it/s, accuracy01=1.000, loss=0.098]
131/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.37it/s, accuracy01=1.000, loss=0.098]

131/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s]
131/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.667, loss=0.660]
131/3000 * Epoch (valid): 100% 1/1 [00:00<00:00, 11.64it/s, accuracy01=0.667, loss=0.660]
[2020-10-02 17:28:12,280] 
131/3000 * Epoch 131 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=0.0976
131/3000 * Epoch 131 (valid): accurac

138/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.40it/s, accuracy01=1.000, loss=0.082]
138/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.29it/s, accuracy01=1.000, loss=0.082]

138/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s]
138/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.800, loss=0.677]
138/3000 * Epoch (valid): 100% 1/1 [00:00<00:00, 10.97it/s, accuracy01=0.800, loss=0.677]
[2020-10-02 17:28:15,572] 
138/3000 * Epoch 138 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=0.0817
138/3000 * Epoch 138 (valid): accuracy01=0.8000 | auc/_mean=0.8013 | auc/class_0=0.8036 | auc/class_1=0.7991 | loss=0.6765
[2020-10-02 17:28:15,572] 
138/3000 * Epoch 138 (train): accuracy01=1.0000 | auc/_mean=1.0000 | auc/class_0=1.0000 | auc/class_1=1.0000 | loss=0.0817
138/3000 * Epoch 138 (valid): accuracy01=0.8000 | auc/_mean=0.8013 | auc/class_0=0.8036 | auc/class_1=0.7991 | loss=0.6765

139/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it

[2020-10-02 17:28:18,663] 
145/3000 * Epoch 145 (train): accuracy01=0.5000 | auc/_mean=0.8867 | auc/class_0=0.8756 | auc/class_1=0.8978 | loss=2.5541
145/3000 * Epoch 145 (valid): accuracy01=0.5333 | auc/_mean=0.4554 | auc/class_0=0.4152 | auc/class_1=0.4955 | loss=1.2636

146/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s]
146/3000 * Epoch (train):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.500, loss=1.329]
146/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  4.04it/s, accuracy01=0.500, loss=1.329]
146/3000 * Epoch (train): 100% 1/1 [00:00<00:00,  3.89it/s, accuracy01=0.500, loss=1.329]

146/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s]
146/3000 * Epoch (valid):   0% 0/1 [00:00<?, ?it/s, accuracy01=0.533, loss=0.926]
146/3000 * Epoch (valid): 100% 1/1 [00:00<00:00, 11.04it/s, accuracy01=0.533, loss=0.926]
[2020-10-02 17:28:19,077] 
146/3000 * Epoch 146 (train): accuracy01=0.5000 | auc/_mean=0.7733 | auc/class_0=0.7600 | auc/class_1=0.7867 | loss=1.3288
146/3000 * Epoch 146 (valid): accurac

Test  Epoch Loss: 0.8343836665153503, Epoch Accuracy: 0.6666666865348816, roc: 0.6830357142857143,  Test


save the results in the required file

In [18]:
np_results = results.numpy()
tresult_csv = os.path.join(args.path, 'test_results' + sID + '.csv')
np.savetxt(tresult_csv, np_results, delimiter=",")
elapsed = time.time() - start_time
print('total time = ', elapsed);

total time =  421.7247989177704


/Users/umahmood1/opt/anaconda3/envs/MILC/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

